# College Scorecard EDA
## Set Up & Loading

In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date

In [2]:
# file path
dataPath = "./"

# file name format MERGEDYYYY_yy_PP.csv

# create a list of tuples with years for filename
# example [('1996', '97'), ('1997', '98'), ...]
yrs = [(date(year = x, month = 1, day = 1).strftime("%Y"),
       date(year = x+1, month = 1, day = 1).strftime("%y")) 
       for x in range(1996,2020)]

# df_dict will map each year to it's dataframe
    # key - 2 digit year
    # value - dataframe for year
    # example: df_dict['05'] returns the dataframe for 2004-05
df_dict = {}

# initialize memory counter
total_memory = 0

for y in yrs:
    start_yr = y[0]
    end_yr = y[1]
    
    # read in dataframe
    df = pd.read_csv(f"MERGED{start_yr}_{end_yr}_PP.csv", low_memory=False)
    
    # add to dictionary
    df_dict.update({end_yr : df})
    
    #sum memory
    total_memory += df.memory_usage(deep = True, index = True).sum()
    
    
print(f"Total Memory Usage: {total_memory/1000000000} GB")

Total Memory Usage: 13.968745186 GB


In [3]:
# check that each dataframe was added to df_dict
df_dict.keys()

dict_keys(['97', '98', '99', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'])

In [4]:
# summarize a dataframe
df_dict['05'].info(verbose = True, memory_usage = 'deep', show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6747 entries, 0 to 6746
Data columns (total 2989 columns):
 #     Column                            Non-Null Count  Dtype  
---    ------                            --------------  -----  
 0     UNITID                            6747 non-null   int64  
 1     OPEID                             6747 non-null   object 
 2     OPEID6                            6747 non-null   int64  
 3     INSTNM                            6747 non-null   object 
 4     CITY                              6747 non-null   object 
 5     STABBR                            6747 non-null   object 
 6     ZIP                               6747 non-null   object 
 7     ACCREDAGENCY                      0 non-null      float64
 8     INSTURL                           0 non-null      float64
 9     NPCURL                            0 non-null      float64
 10    SCH_DEG                           6570 non-null   float64
 11    HCM2                              0 non

---
We have added each dataframe to a dictionary, indexable by year (97-20)

In [5]:
'''
This block of code compares the columns in each
dataframe. The goal was to create a list of unique 
columns for each dataframe. We learned that the 
columns are the same in each dataframe. So, 
the column lists are empty
'''

df_cols = {}
for k, df in df_dict.items():
    df_cols.update({k : set(df.columns)})

# create union of columns for each df
total_set = set()
for k, s in df_cols.items():
    if k == '97':
        total_set = s
    else:
        total_set = total_set.union(s)
        
# compare each df column set across total column set
differing_cols = {}
for k, s in df_cols.items():
    # collect everything in the total set that is not in s
    tot_minus_s = total_set.difference(s)
    
    # collect everything in s that is no in total set
    s_minus_tot = s.difference(total_set)
    
    # take union
    differ_union = tot_minus_s.union(s_minus_tot)
    differing_cols.update({k : differ_union})
    
differing_cols

{'97': set(),
 '98': set(),
 '99': set(),
 '00': set(),
 '01': set(),
 '02': set(),
 '03': set(),
 '04': set(),
 '05': set(),
 '06': set(),
 '07': set(),
 '08': set(),
 '09': set(),
 '10': set(),
 '11': set(),
 '12': set(),
 '13': set(),
 '14': set(),
 '15': set(),
 '16': set(),
 '17': set(),
 '18': set(),
 '19': set(),
 '20': set()}

For each dataframe, collect a set of completely null columns. Take the intersection of these sets to find total set of completely null columns. These columns can be dropped immediately since we have no data from any year.

In [6]:
# This is super slow!!!

'''
null_dict is a dictionary that maps a year to its
set of completely null columns
    key - 2 digit year
    value - set of null columns
'''
null_dict = {}

for k, df in df_dict.items():
    null_cols = set()
    for c in df.columns:
        if df[c].notna().sum() == 0:
            null_cols.add(c)
    
    null_dict.update({k : null_cols})

In [7]:
# get count of null cols per df
for k, n in null_dict.items():
    print(f"Year {k} number of null columns: {len(n)}")

Year 97 number of null columns: 2708
Year 98 number of null columns: 2575
Year 99 number of null columns: 2388
Year 00 number of null columns: 2175
Year 01 number of null columns: 1992
Year 02 number of null columns: 1954
Year 03 number of null columns: 1770
Year 04 number of null columns: 1729
Year 05 number of null columns: 1563
Year 06 number of null columns: 1519
Year 07 number of null columns: 1547
Year 08 number of null columns: 1466
Year 09 number of null columns: 1472
Year 10 number of null columns: 1312
Year 11 number of null columns: 1370
Year 12 number of null columns: 1330
Year 13 number of null columns: 1341
Year 14 number of null columns: 1307
Year 15 number of null columns: 1260
Year 16 number of null columns: 1298
Year 17 number of null columns: 1297
Year 18 number of null columns: 1300
Year 19 number of null columns: 1099
Year 20 number of null columns: 1421


In [8]:
'''
total_null_cols is a set of columns that are
completely null in the entire dataset.
There are only 11 such columns.
'''
total_null_cols = set()
for k, n in null_dict.items():
    if k == '97':
        total_null_cols = n
    else:
        total_null_cols = total_null_cols.intersection(n)
        
# convert to list
#total_null_cols = list(total_null_cols)

# print number of completely
print(f"There are {len(total_null_cols)} completely null columns.")

There are 11 completely null columns.


Sadly, there are only 11 completely null columns
+ `AGE_ENTRY_SQ` - average of `AGE` squared
+ `COUNT_ED` - count of student in earnings cohort (this seems important?)
+ `FSEND_1` - Share of students who sent FAFSA to only 1 college
+ `FSEND_2`
+ `FSEND_3`
+ `FSEND_4`
+ `FSEND_5`
+ `FSEND_COUNT` - Average number of students who sent FAFSA to at least 1 college
+ `LNFAMINC` - Average of log of family income
+ `LNFAMINC_IND` - Average of log of family income for independent students
+ `LOCALE2` - Degree of urbanization of institution

I drop these columns now, even though this is only a modest improvement in memory size.

In [9]:
# How  much memory is saved by dropping 11 cols?
total_memory2 = 0

for k, df in df_dict.items():
    df_temp = df.drop(columns = total_null_cols)
    #df_dict.update({k : df})
    
    # compute memory size now
    total_memory2 += df_temp.memory_usage(deep = True, index = True).sum()
    
print(f"Total Memory Usage: {total_memory2/1000000000} GB")
print(f"Memory reduction: {(1-(total_memory2/total_memory))*100}%")

Total Memory Usage: 13.953782898 GB
Memory reduction: 0.10711261320018872%


We only dropped the intersection of completely null columns. What if we drop the union?

In [10]:
'''
union_null_cols is the set of columns
that are completely null in any given dataframe.
'''
union_null_cols = set()
for k, n in null_dict.items():
    if k == '97':
        union_null_cols = n
    else:
        union_null_cols = union_null_cols.union(n)
        
# convert to list
#total_null_cols = list(total_null_cols)

# print number of completely
print(f"There are {len(union_null_cols)} elements in the union of completely null columns.")

There are 2724 elements in the union of completely null columns.


In [11]:
# compute memory reduction of dropping union of null columns
total_memory3 = 0

for k, df in df_dict.items():
    df_temp = df.drop(columns = union_null_cols)
    #df_dict.update({k : df})
    
    # compute memory size now
    total_memory3 += df_temp.memory_usage(deep = True, index = True).sum()
    
print(f"Total Memory Usage: {total_memory3/1000000000} GB")
print(f"Memory reduction: {(1-(total_memory3/total_memory))*100}%")

Total Memory Usage: 0.444904798 GB
Memory reduction: 96.81499811131282%


In [12]:
# compute nullness of dataframes with remaining columns
for k, df in df_dict.items():
    df_temp = df.drop(columns = union_null_cols)
    nullness = df_temp.isna().sum().sum()/df_temp.size
    print(f"For year {k}, the nullness is {round(nullness*100,2)}%")

For year 97, the nullness is 10.28%
For year 98, the nullness is 10.66%
For year 99, the nullness is 7.97%
For year 00, the nullness is 7.38%
For year 01, the nullness is 7.51%
For year 02, the nullness is 7.77%
For year 03, the nullness is 7.82%
For year 04, the nullness is 6.82%
For year 05, the nullness is 8.26%
For year 06, the nullness is 9.25%
For year 07, the nullness is 9.21%
For year 08, the nullness is 9.28%
For year 09, the nullness is 9.18%
For year 10, the nullness is 9.61%
For year 11, the nullness is 9.26%
For year 12, the nullness is 9.77%
For year 13, the nullness is 10.69%
For year 14, the nullness is 10.97%
For year 15, the nullness is 11.02%
For year 16, the nullness is 11.38%
For year 17, the nullness is 11.6%
For year 18, the nullness is 11.76%
For year 19, the nullness is 12.42%
For year 20, the nullness is 12.89%


In [19]:
df_dict['05'].drop(columns = union_null_cols).info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6747 entries, 0 to 6746
Data columns (total 265 columns):
 #    Column      Non-Null Count  Dtype  
---   ------      --------------  -----  
 0    UNITID      6747 non-null   int64  
 1    OPEID       6747 non-null   object 
 2    OPEID6      6747 non-null   int64  
 3    INSTNM      6747 non-null   object 
 4    CITY        6747 non-null   object 
 5    STABBR      6747 non-null   object 
 6    ZIP         6747 non-null   object 
 7    MAIN        6747 non-null   int64  
 8    NUMBRANCH   6747 non-null   int64  
 9    PREDDEG     6747 non-null   int64  
 10   HIGHDEG     6747 non-null   int64  
 11   CONTROL     6747 non-null   int64  
 12   ST_FIPS     6747 non-null   int64  
 13   REGION      6747 non-null   int64  
 14   PCIP01      6313 non-null   float64
 15   PCIP03      6313 non-null   float64
 16   PCIP04      6313 non-null   float64
 17   PCIP05      6313 non-null   float64
 18   PCIP09      6313 non-null   float64
 19   PCIP

---
There are still some significant issues with nullness. Next, I want to investigate nullness over the rows (i.e. institutions). Maybe I can create a subset of institutions that are mostly populated.

Create union set and intersection set of institutions in each dataframe. This is potentially challeninging if institution names differe between dataframes.



In [13]:
'''
inst_union is the set of institutions that ever appear in the dataset.
inst_intersection is the set of institutions that appear every year in the dataset.
'''

inst_union = set()
inst_intersection = set()

for k, df in df_dict.items():
    inst_set = set(df['INSTNM'])
    if k == '97':
        inst_union = set(inst_set)
        inst_intersection = set(inst_set)
    else:
        inst_union = inst_union.union(inst_set)
        inst_intersection = inst_intersection.intersection(inst_set)
        
print(f"Total number of institutions in dataset {len(inst_union)}")
print(f"Number of institutions present every year {len(inst_intersection)}")

Total number of institutions in dataset 11207
Number of institutions present every year 4066


In [14]:
'''
Lets compute memory reduction by keeping only institutions
that persist over the entire time period.
'''
total_memory4 = 0
for k, df in df_dict.items():
    df_temp = df[df['INSTNM'].isin(inst_intersection)]
    
    # compute memory size now
    total_memory4 += df_temp.memory_usage(deep = True, index = True).sum()
    
print(f"Total Memory Usage: {total_memory4/1000000000} GB")
print(f"Memory reduction: {(1-(total_memory4/total_memory))*100}%")

Total Memory Usage: 8.237210784 GB
Memory reduction: 41.0311329019328%


In [15]:
'''
Compute the nullness of each dataframe
when only the 4066 institutions are kept
'''
nullness = 0
for k, df in df_dict.items():
    df_temp = df[df['INSTNM'].isin(inst_intersection)]
    nullness = df_temp.isna().sum().sum()/df_temp.size
    print(f"For year {k}, the nullness is {round(nullness*100,2)}%")

For year 97, the nullness is 91.43%
For year 98, the nullness is 87.84%
For year 99, the nullness is 81.7%
For year 00, the nullness is 74.75%
For year 01, the nullness is 68.82%
For year 02, the nullness is 68.08%
For year 03, the nullness is 62.03%
For year 04, the nullness is 60.68%
For year 05, the nullness is 55.34%
For year 06, the nullness is 53.87%
For year 07, the nullness is 55.06%
For year 08, the nullness is 52.47%
For year 09, the nullness is 53.73%
For year 10, the nullness is 50.19%
For year 11, the nullness is 52.08%
For year 12, the nullness is 49.37%
For year 13, the nullness is 49.71%
For year 14, the nullness is 48.73%
For year 15, the nullness is 47.21%
For year 16, the nullness is 48.95%
For year 17, the nullness is 48.44%
For year 18, the nullness is 48.56%
For year 19, the nullness is 42.38%
For year 20, the nullness is 56.84%


**Next Steps**
Using the subset of institutions that are present throughout, repeat the column reduction procedure. Are we left with more columns?

----
**To Do**
+ ~~Check that the columns are the same across all DFs~~
    + If columns are standardized, then combine into single DF with additional column for year
        + Do I need to reduce size before combining? 
    + If not the same, get list of differing columns
+ ~~Identify which columns are completely null for every year 1997-2020~~
+ ~~Identify which columns are completely null for some years~~



+ Look at last 10 years
    + Keep only universities that are present every year
    + Drop columns that are always null
    + Look at columns which are sometimes null. Decide whether to keep or drop
+ How does pd.read_csv() treat 'PrivacySupressed'
+ *For Brooks:* Drop universities, then investigate columns for 1997-2020

**Question**
+ When does our dataset need to be <3 GB?